# Tutorial como instalar Apache Airflow com Docker

## O que é o Apache Airflow?

É uma plataforma de código aberto desenvolvida pela comunidade para criar, agendar e monitorar workflow de maneira automática. 

É normalmente utilizado quando temos fluxos de trabalho computacionais complexos e pipelines de processamento de dados. 

Podemos ver como um agendador de tarefas, onde uma tarefa pode consistir em várias etapas com dependências complexas. Com isso o Airflow executa as tarefas em horários predefinidos, seguindo as dependências, determinando a ordem de execução das tarefas, quais tarefas são dependentes umas das outras e quais tarefas devem ser acionadas.

## Quais vantagens de usar?

**Dinâmico**: Os DAGs no Airflow são completamente dinâmicos e flexíveis. Os pipelines do Airflow são configurados como código (Python), permitindo a geração dinâmica de pipelines e proporcionando controle total sobre eles.

**Extensível**: O Airflow oferece diversos executores e operadores que suportam praticamente todos os tipos de execução de tarefas. É possível definir facilmente seus próprios operadores e executores, estendendo a biblioteca para adequá-la ao nível de abstração que se adapta ao seu ambiente.

**Escalabilidade**: A capacidade de medir quantas tarefas podem ser executadas é suportada por sistemas de múltiplos nós de trabalhadores, permitindo a implantação de um número variável de nós de trabalho.

## O que são as DAGS?

O workflow é feito a partir das Directed Acyclic Graphs (DAGS), que nada mais são que sequências de tarefas a serem executadas.

Pensndo em uma estrutura de grafos, uma DAG é um nó que conecta as arestas, onde cada nó no grafo representa uma tarefa, e as arestas definem as dependências entre as tarefas.

Exemplos de tarefas: 
- Transferência de dados de uma tarefa para outra.
- Realização de backups.
- Automatização de operações.
- Experimentos de aprendizado de máquina.

O que é uma Tarefa (Task)?
A tarefa é representada como um nó na DAG. No exemplo abaixo, wash_rice é uma tarefa, assim como os outros nós. Perceba que a task cook_rice só será executada uma vez que wash_rice foi executada corretamente. Assim como roll_sushi só será executado quando todas as anteriores tiverem terminado.


<img src="imgs/sushi-roll-workflow-1.png" width="600" height="400">



## E os Operadores?

Um operador determina o que vai ser feito. Você pode pensar em um operador como um objeto ou tarefa dentro do DAG, e existem muitos tipos de operadores que vamos explicar:

## Componentes

Diagrama das principais componentes:

![alt text](imgs/comps.png "Title")


**Tipos de Executors (Executores):**
Quando se trata de executar tarefas no Apache Airflow, existem vários tipos de executores disponíveis. Cada executor tem seus próprios pontos fortes e fracos, dependendo do caso de uso específico. Nesta seção, exploraremos quatro tipos comuns de executores: 

**Sequential Executor (Executor Sequencial):**
O executor sequencial é o executor padrão. Executa uma instância de tarefa por vez. Aqui, o banco de dados de metadados pode ser SQLite. 

Prós:
- Nenhuma configuração é necessária. 
- Leve e econômico.

Contras:
- Não escalável.
- Lento, pois executa uma tarefa de cada vez.
- Ponto único de falha.
- Não adequado para produção.

**Local Executor (Executor Local):**
Executa com o agendador, pode executar várias tarefas simultaneamente, e aqui, o banco de dados de metadados pode ser MYSQL ou PostgreSQL.

Prós:
- Fácil de configurar.
- Leve.
- Pode executar várias tarefas

Contras:
* Ponto único de falha.
* Não adequado para produção.


**Celery Executor (Executor Celery):**
Executa tarefas em uma máquina dedicada, consulta de tarefas distribuída, e aqui, o banco de dados de metadados pode ser MYSQL ou PostgreSQL. A máquina continuará funcionando mesmo se não houver tarefas em execução. Por esse motivo, o Celery é usado se tivermos um alto volume de tarefas para evitar desperdício de recursos.

Prós:
- Tolerante a falhas.
- Adequado para ambientes de produção.

Contras:
- Projetado para escala horizontal.
* Demora para configurar.
* Desperdício de recursos.
* Não é economicamente eficiente.

**Kubernetes Executor (Executor Kubernetes):**
Usa a API do Kubernetes para gerenciar pods e você pode atribuir recursos com base no seu uso.

Prós:
- Pode atribuir recursos a tarefas individuais.
- Eficiente em termos de custo e recursos.

Contras:
- Requer conhecimento em Kubernetes.

# Instalação Via Docker

Siga estas etapas para instalar as ferramentas necessárias, caso ainda não o tenha feito.

Instale o Docker Community Edition (CE).

Instale o Docker Compose v2.14.0 ou uma versão mais recente.

In [16]:
!docker compose version

!docker --version

Docker Compose version v2.21.0
Docker version 24.0.7, build afdd53b


Vamos baixar o docker-compose

In [19]:
!curl -LfO 'https://airflow.apache.org/docs/apache-airflow/2.8.0/docker-compose.yaml'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10940  100 10940    0     0  57249      0 --:--:-- --:--:-- --:--:-- 57277


Vamos ter vários serviços:
**Postgres**: Executa um banco de dados PostgreSQL, comumente usado como o banco de dados principal do Apache Airflow. Armazena metadados e logs de execução de tarefas.

**Redis**: Executa um servidor Redis, frequentemente usado como o broker de mensagens para o Apache Airflow. O scheduler e o worker utilizam o Redis para comunicação e coordenação da execução de tarefas.

**airflow-webserver**: Executa o servidor web do Apache Airflow, a principal interface para interagir com o Apache Airflow. Fornece uma UI amigável para gerenciar DAGs, cronogramas e tarefas.

**airflow-scheduler**: Executa o scheduler do Apache Airflow, responsável por agendar tarefas e acionar instâncias de tarefas para execução. Verifica novas tarefas a serem executadas e cria instâncias com base no cronograma definido.

**airflow-worker**: Executa o worker do Apache Airflow, que executa tarefas definidas nos DAGs do Apache Airflow. O worker aguarda novas tarefas e as executa quando acionado pelo scheduler.

**airflow-triggerer**: Executa um processo separado que escuta por acionadores externos para iniciar DAGs do Apache Airflow. Isso pode ser útil para acionar fluxos de trabalho com base em eventos ou dados externos.

**airflow-init**: Executa scripts de inicialização para configurar o esquema do banco de dados do Airflow e criar um usuário administrador. Normalmente, é executado apenas uma vez durante a configuração inicial do ambiente do Apache Airflow.

**flower: Executa um processo separado que fornece uma interface web para monitorar tarefas e workers do Apache Airflow. Pode ser útil para acompanhar o progresso das tarefas e solucionar problemas.

Vamos criar as pastas que serão compartilhadas entre o o seu computador e o container.

./dags: Você pode colocar seus arquivos DAG aqui.

./logs: Contém logs da execução de tarefas e do scheduler.

./plugins: Você pode colocar seus plugins aqui.

./config: Arquivos de configuração ficam aqui.

In [22]:
!mkdir -p ./dags ./logs ./plugins ./config

A instância do Apache Airflow pode ser inicializada agora utilizando o serviço 'airflow-init'.

In [1]:
!docker compose up airflow-init


Agora que todos os serviços relacionados foram inicializados, é hora de executar o Airflow.

In [2]:
!docker-compose up -d

Para verificar se o serviço do Airflow está em execução, execute o seguinte comando no prompt de comando

In [5]:
!docker ps

CONTAINER ID   IMAGE                  COMMAND                  CREATED              STATUS                             PORTS                                       NAMES
6ab2890ea98e   apache/airflow:2.8.0   "/usr/bin/dumb-init …"   28 seconds ago       Up 26 seconds (health: starting)   8080/tcp                                    airflow_airflow-scheduler_1
8461d2abf916   apache/airflow:2.8.0   "/usr/bin/dumb-init …"   28 seconds ago       Up 26 seconds (health: starting)   8080/tcp                                    airflow_airflow-triggerer_1
0d479b9d0fce   apache/airflow:2.8.0   "/usr/bin/dumb-init …"   28 seconds ago       Up 25 seconds (health: starting)   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp   airflow_airflow-webserver_1
18d20622a0b7   apache/airflow:2.8.0   "/usr/bin/dumb-init …"   28 seconds ago       Up 26 seconds (health: starting)   8080/tcp                                    airflow_airflow-worker_1
bf4bb5c7e8bc   postgres:13            "docker-entrypoint.s…"   About a


Abra um navegador e digite a URL local: http://localhost:8080. Prrencha o nome de usuário como "airflow" e a senha como "airflow".

Finalmente, o Apache Airflow está em execução!

![alt text](imgs/capturas.png "Title")
